In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

import pyarrow as pa
import pyarrow.parquet as pq

import statsmodels.formula.api as smf
from linearmodels.iv import IV2SLS
from linearmodels.panel import PanelOLS

from census import Census # This is new...

C:\ProgramData\Anaconda3\lib\site-packages\linearmodels\panel\data.py:10: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import (Categorical, DataFrame, Index, MultiIndex, Panel, Series,


In [2]:
os.get_exec_path

<function os.get_exec_path(env=None)>

In [3]:
cwd = os.getcwd()

trade_data = pq.read_table(cwd+ "\\data\\total_trade_data_2015.parquet").to_pandas()

trade_data["time"] = pd.to_datetime(trade_data.time)

trade_data.set_index(["area_fips", "time"],inplace = True)

In [4]:
trade_data.head()

exposure = pd.qcut(trade_data.xs('2018-12-1', level=1).tariff, 4 ,labels = False)

most_exposed = exposure[exposure == 3].index.tolist()

trade_data.loc[most_exposed].xs('2018-12-1', level=1).tariff.mean()

6.4691811550724125

In [5]:
years = [16,17,18,19]

empl_time_dict = {}

foo = 0

for xxx in years:
    
    
    year = int("20" + str(xxx))
    
    
    empl_time_dict["year_{0}".format(year)] = {"1_month1_emplvl":dt.datetime(year,1,1),
                 "1_month2_emplvl":dt.datetime(year,2,1),
                 "1_month3_emplvl":dt.datetime(year,3,1),
                 "2_month1_emplvl":dt.datetime(year,4,1),
                 "2_month2_emplvl":dt.datetime(year,5,1),
                 "2_month3_emplvl":dt.datetime(year,6,1),
                 "3_month1_emplvl":dt.datetime(year,7,1),
                 "3_month2_emplvl":dt.datetime(year,8,1),
                 "3_month3_emplvl":dt.datetime(year,9,1),
                 "4_month1_emplvl":dt.datetime(year,10,1),
                 "4_month2_emplvl":dt.datetime(year,11,1),
                 "4_month3_emplvl":dt.datetime(year,12,1),}

In [6]:
print("")
print("**********************************************************************************")
print("Downloading and processing 2019 BLS file")
print("")

url = "https://data.bls.gov/cew/data/files/2019/csv/2019_qtrly_singlefile.zip"
# This will read in the annual, single file. It's big, but has all we want...

r = requests.get(url) 

# convert bytes to zip file  
bls_q2019 = zf.ZipFile(io.BytesIO(r.content)) 
bls_q2019.extractall(cwd + "\\bls_single_files")

In [68]:
print("")
print("**********************************************************************************")
print("Downloading and processing 2018 BLS file")
print("")

url = "https://data.bls.gov/cew/data/files/2018/csv/2018_qtrly_singlefile.zip"
# This will read in the annual, single file. It's big, but has all we want...

r = requests.get(url) 

# convert bytes to zip file  
bls_q2018 = zf.ZipFile(io.BytesIO(r.content)) 
bls_q2018.extractall(cwd + "\\bls_single_files")

In [177]:
print("")
print("**********************************************************************************")
print("Downloading and processing 2017 BLS file")
print("")

url = "https://data.bls.gov/cew/data/files/2017/csv/2017_qtrly_singlefile.zip"
# This will read in the annual, single file. It's big, but has all we want...

r = requests.get(url) 

# convert bytes to zip file  
bls_q2017 = zf.ZipFile(io.BytesIO(r.content)) 
bls_q2017.extractall(cwd + "\\bls_single_files")

In [180]:
print("")
print("**********************************************************************************")
print("Downloading and processing 2016 BLS file")
print("")

url = "https://data.bls.gov/cew/data/files/2016/csv/2016_qtrly_singlefile.zip"
# This will read in the annual, single file. It's big, but has all we want...

r = requests.get(url) 

# convert bytes to zip file  
bls_q2016 = zf.ZipFile(io.BytesIO(r.content)) 
bls_q2016.extractall(cwd + "\\bls_single_files")

In [7]:
def bls_quarter_cat(df,agg_cat,cat,qrt,time_dict,var_name):
    
    # Take only private
    df = df[df["qtr"] == int(qrt)] 
    
    df = df[df["own_code"] == 5] 

# Take aggregate

    df = df[df["agglvl_code"] == agg_cat] 
    
    df = df[df["industry_code"] == cat] # Take all employment in all sectors
    
    rename_months = {"month1_emplvl": str(qrt) + "_month1_emplvl",
             "month2_emplvl": str(qrt) + "_month2_emplvl",
             "month3_emplvl": str(qrt) + "_month3_emplvl"}
    
    df.rename(rename_months, axis = 1, inplace = True)

# Take only counties 
    
    df.rename({"area_fips": "GEOFIPS"},axis = 1, inplace = True)

    df["GEOFIPS"] = df["GEOFIPS"].astype(int)

    df.set_index("GEOFIPS", inplace = True)

    df = df.reindex(trade_data.index.get_level_values(0).unique().astype(int).tolist())
    # this part makes sure that we are always getting same counties as in the trade data set
    # if we don't do this, things are floating around. The issue I think is for some of the really
    # small counties, the employment in the catagory is withehld, so when we grab this stuff,
    # the county goes missing

    df = df.iloc[:,[4,5,6]].reset_index()
    # This grabs only values we want, i.e. the employment for that quarter. So for example,
    # in Q1, 13 = January, 14 = Febuary, 15 = March. And so forth for Q2...

    df = df.melt("GEOFIPS")

    df.replace(time_dict,inplace = True)

    df.rename({"variable":"time", "value": var_name, "GEOFIPS": "area_fips"}, axis = 1, inplace = True)
    
    df["area_fips"] = df["area_fips"].astype(str)
    
    df.set_index(["area_fips", "time"], inplace = True)
    
    #df.sort_values(["area_fips", "time"], inplace = True)
    
    return df

In [8]:
def clean_bls_quarter(code,agg_cat,var_name):

############################################################################  
#code = "44-45"
#agg_cat = 74
#71 and 10 gets you total
# 72 and 101 and 102 gets you goods, no goods
# 74 and 44-45 gets you retail trade

# Need an approach to get all the stuff in one file. 

    df = pd.DataFrame([])
    quarter = ["1","2","3","4"]

############################################################################

    #file_name = "C:\\github\\consumption_and_tradewar\\sandbox\\bls_single_files\\2016.q1-q4.singlefile.csv"
    file_name = cwd + "\\bls_single_files\\2016.q1-q4.singlefile.csv"

    df_year = pd.read_csv(file_name, usecols= ["area_fips","own_code","industry_code","agglvl_code",
                                     'month1_emplvl','month2_emplvl', 'month3_emplvl',"qtr"])

    for item in quarter:
        
        df = df.append(bls_quarter_cat(df_year,agg_cat, code , item, empl_time_dict["year_2016"], var_name))

############################################################################

    #file_name = "C:\\github\\consumption_and_tradewar\\sandbox\\bls_single_files\\2017.q1-q4.singlefile.csv"
    file_name = cwd + "\\bls_single_files\\2017.q1-q4.singlefile.csv"
    
    df_year = pd.read_csv(file_name, usecols= ["area_fips","own_code","industry_code","agglvl_code",
                                     'month1_emplvl','month2_emplvl', 'month3_emplvl',"qtr"])

    for item in quarter:
        
        df = df.append(bls_quarter_cat(df_year,agg_cat, code , item, empl_time_dict["year_2017"], var_name))


############################################################################

    #file_name = "C:\\github\\consumption_and_tradewar\\sandbox\\bls_single_files\\2018.q1-q4.singlefile.csv"
    file_name = cwd + "\\bls_single_files\\2018.q1-q4.singlefile.csv"
    
    df_year = pd.read_csv(file_name, usecols= ["area_fips","own_code","industry_code","agglvl_code",
                                     'month1_emplvl','month2_emplvl', 'month3_emplvl',"qtr"])

    for item in quarter:
        
        df = df.append(bls_quarter_cat(df_year,agg_cat, code , item, empl_time_dict["year_2018"], var_name))
    
############################################################################

    #file_name = "C:\\github\\consumption_and_tradewar\\sandbox\\bls_single_files\\2019.q1-q2.singlefile.csv"
    file_name = cwd + "\\bls_single_files\\2019.q1-q2.singlefile.csv"
    
    df_year = pd.read_csv(file_name, usecols= ["area_fips","own_code","industry_code","agglvl_code",
                                     'month1_emplvl','month2_emplvl', 'month3_emplvl',"qtr"])
    
    quarter = ["1","2"]

    for item in quarter:
    
        df = df.append(bls_quarter_cat(df_year,agg_cat, code , item, empl_time_dict["year_2019"], var_name))
        
    df.sort_values(["area_fips", "time"], inplace = True)
    
    return df

In [9]:
print("")
print("**********************************************************************************")
print("Constructing Employment Series")
print("")

code = "44-45"
agg_cat = 74

df_rtl = clean_bls_quarter(code,agg_cat,"emp_rtl")

#####################################################################33
#71 and 10 gets you total

code = "10"
agg_cat = 71

df_all = clean_bls_quarter(code,agg_cat,"emp_all")

#####################################################################33
#71 and 10 gets you total

code = "101"
agg_cat = 72

df_gds = clean_bls_quarter(code,agg_cat,"emp_gds")

#####################################################################33
#71 and 10 gets you total

code = "102"
agg_cat = 72

df_ngds = clean_bls_quarter(code,agg_cat,"emp_ngds")

########################################################################

df = df_rtl.merge(df_all, left_index = True, right_index = True)

df = df.merge(df_gds, left_index = True, right_index = True)

df = df.merge(df_ngds, left_index = True, right_index = True)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [10]:
df.head(50)

emp_rtl   emp_all  emp_gds  emp_ngds
area_fips time                                            
10001     2016-01-01   9269.0       0.0      0.0   38494.0
          2016-02-01   9236.0       0.0      0.0   38646.0
          2016-03-01   9342.0       0.0      0.0   38917.0
          2016-04-01   9376.0       0.0      0.0   39719.0
          2016-05-01   9265.0       0.0      0.0   40164.0
          2016-06-01   9345.0       0.0      0.0   39656.0
          2016-07-01   9441.0       0.0      0.0   39773.0
          2016-08-01   9495.0       0.0      0.0   39746.0
          2016-09-01   9372.0       0.0      0.0   39881.0
          2016-10-01   9509.0       0.0      0.0   40038.0
          2016-11-01   9859.0       0.0      0.0   40254.0
          2016-12-01   9762.0       0.0      0.0   40220.0
          2017-01-01   9463.0       0.0      0.0   39246.0
          2017-02-01   9458.0       0.0      0.0   39490.0
          2017-03-01   9453.0       0.0      0.0   39598.0
          2017-04-01   9482.0       0.0      0.0   39804.0
          2017-05-01   9498.0       0.0      0.0   39950.0
          2017-06-01   9519.0       0.0      0.0   40318.0
          2017-07-01   9410.0       0.0      0.0   39670.0
          2017-08-01   9403.0       0.0      0.0   39467.0
          2017-09-01   9496.0       0.0      0.0   40023.0
          2017-10-01   9441.0       0.0      0.0   39791.0
          2017-11-01   9927.0       0.0      0.0   40512.0
          2017-12-01   9858.0       0.0      0.0   40292.0
          2018-01-01   9336.0       0.0      0.0   39580.0
          2018-02-01   9248.0       0.0      0.0   39479.0
          2018-03-01   9341.0       0.0      0.0   39829.0
          2018-04-01   9402.0       0.0      0.0   40631.0
          2018-05-01   9489.0       0.0      0.0   41179.0
          2018-06-01   9486.0       0.0      0.0   41147.0
          2018-07-01   9348.0       0.0      0.0   39952.0
          2018-08-01   9314.0       0.0      0.0   39830.0
          2018-09-01   9240.0       0.0      0.0   40133.0
          2018-10-01   9310.0       0.0      0.0   40562.0
          2018-11-01   9557.0       0.0      0.0   40809.0
          2018-12-01   9529.0       0.0      0.0   40748.0
          2019-01-01   9287.0       0.0      0.0   40347.0
          2019-02-01   9225.0       0.0      0.0   40337.0
          2019-03-01   9200.0       0.0      0.0   40576.0
          2019-04-01   9195.0       0.0      0.0   41241.0
          2019-05-01   9238.0       0.0      0.0   41666.0
          2019-06-01   9336.0       0.0      0.0   41439.0
10003     2016-01-01  30792.0  247839.0  23622.0  224217.0
          2016-02-01  30470.0  246561.0  23481.0  223080.0
          2016-03-01  30792.0  248144.0  24015.0  224129.0
          2016-04-01  31029.0  250501.0  24094.0  226407.0
          2016-05-01  31213.0  251305.0  24257.0  227048.0
          2016-06-01  31276.0  252955.0  24598.0  228357.0
          2016-07-01  31306.0  253356.0  25067.0  228289.0
          2016-08-01  31358.0  252479.0  25191.0  227288.0

In [11]:
trade_employ = trade_data.merge(df, left_index = True, right_index = True, how = "right")
# This is a place to be mindfull about time period, if we want 
# do left if you just want to conform with the trade data

In [12]:
trade_employ.head(50)

total_exp_pc  china_exp_pc    tariff  emplvl_2017  \
area_fips time                                                            
10001     2016-01-01    453.257185     47.280196  1.069532       2843.0   
          2016-02-01    471.930726     47.211522  1.069499       2843.0   
          2016-03-01    485.376760     35.078484  1.069500       2843.0   
          2016-04-01    460.259354     27.991526  1.069500       2843.0   
          2016-05-01    473.572638     28.235163  1.069499       2843.0   
          2016-06-01    476.828261     28.236923  1.069499       2843.0   
          2016-07-01    481.979929     36.513882  1.069499       2843.0   
          2016-08-01    501.651039     41.120917  1.069499       2843.0   
          2016-09-01    495.664401     42.858366  1.069499       2843.0   
          2016-10-01    559.415478     86.841250  1.069499       2843.0   
          2016-11-01    558.631462     84.709850  1.069499       2843.0   
          2016-12-01    528.952068     71.799888  1.069499       2843.0   
          2017-01-01    522.580597     58.368700  1.068759       2843.0   
          2017-02-01    502.309185     48.422056  1.068802       2843.0   
          2017-03-01    571.161328     42.286424  1.068895       2843.0   
          2017-04-01    509.278875     33.702476  1.068895       2843.0   
          2017-05-01    519.176693     33.878036  1.068895       2843.0   
          2017-06-01    509.383926     30.102481  1.068895       2843.0   
          2017-07-01    492.508803     32.581600  1.068895       2843.0   
          2017-08-01    509.577144     35.329198  1.068895       2843.0   
          2017-09-01    495.123610     44.380139  1.068895       2843.0   
          2017-10-01    569.787487     73.830992  1.068895       2843.0   
          2017-11-01    571.520102     73.114154  1.068895       2843.0   
          2017-12-01    553.649765     59.845960  1.068895       2843.0   
          2018-01-01    516.700915     48.624283  1.068805       2843.0   
          2018-02-01    516.145992     44.607737  1.068897       2843.0   
          2018-03-01    587.864923     42.979806  1.068894       2843.0   
          2018-04-01    559.410137     36.213274  1.393797       2843.0   
          2018-05-01    579.008640     33.254994  1.393798       2843.0   
          2018-06-01    558.919118     28.428818  1.393798       2843.0   
          2018-07-01    524.597360     25.288644  2.221774       2843.0   
          2018-08-01    545.616139     25.092559  2.221774       2843.0   
          2018-09-01    511.239593     21.133593  2.235515       2843.0   
          2018-10-01    562.324040     24.854552  2.485311       2843.0   
          2018-11-01    555.172968     25.784350  2.426361       2843.0   
          2018-12-01    524.630933     23.745690  2.426361       2843.0   
          2019-01-01    527.484006     29.783300  2.424109       2843.0   
          2019-02-01    505.126798     33.942512  2.424092       2843.0   
          2019-03-01    573.829215     35.695393  2.424058       2843.0   
          2019-04-01    541.332336     27.687231  2.424058       2843.0   
          2019-05-01    553.302909     33.524619  2.424057       2843.0   
          2019-06-01    526.449004     37.436249  2.424056       2843.0   
10003     2016-01-01    430.048151     29.819622  0.211465       9072.0   
          2016-02-01    433.446708     28.284932  0.211465       9072.0   
          2016-03-01    479.220412     36.789866  0.211466       9072.0   
          2016-04-01    454.820218     33.759979  0.211466       9072.0   
          2016-05-01    447.341372     34.152770  0.211466       9072.0   
          2016-06-01    463.404573     33.126661  0.211466       9072.0   
          2016-07-01    442.881011     32.788082  0.211466       9072.0   
          2016-08-01    460.392618     33.688284  0.211466       9072.0   

                       fips  total_employment  emp_rtl   emp_all  emp_gds  \
area_fips time                              

In [13]:
trade_employ.total_employment.fillna(method='bfill', inplace = True)

trade_employ.tariff.fillna(method='bfill', inplace = True)

In [14]:
dfrural = pd.read_excel("https://www2.census.gov/geo/docs/reference/ua/County_Rural_Lookup.xlsx", skiprows=[0,1,2],
                       nrows = 3142)

dfrural["area_fips"] = dfrural["2015 GEOID"].astype(int)

trade_employ.reset_index(inplace = True)

trade_employ["area_fips"] = trade_employ["area_fips"].astype(int)

trade_employ = trade_employ.merge(dfrural[["area_fips", "2010 Census \nPercent Rural",
                             "2010 Census Total Population"]], left_on = "area_fips", right_on = "area_fips",
                                 how = "left")

trade_employ.set_index(["area_fips", "time"],inplace = True)

trade_employ.rename({"2010 Census \nPercent Rural": "rural_share",
            "2010 Census Total Population": "2010_population"}, inplace = True, axis = 1)

trade_employ["rural_share"] = 0.01*trade_employ["rural_share"]

In [15]:
my_api_key = '34e40301bda77077e24c859c6c6c0b721ad73fc7'
# This is my api_key

c = Census(my_api_key)
# This will create an object c which has methods associated with it.
# We will see  these below.

type(c) 
# Per the discussion below, try c.tab and see the options. 

code = ("NAME","B01001_001E","B19013_001E") # Same Codes:

county_2017 = pd.DataFrame(c.acs5.get(code, 
                                         {'for': 'county:*'}, year=2017))
                                         # Same deal, but we specify county then the wild card
                                         # On the example page, there are ways do do this, only by state
        
county_2017 = county_2017.rename(columns = {"B01001_001E":"2017_population", "B19013_001E":"2017_income"})

county_2017["GEOFIPS"] = (county_2017["state"] + county_2017["county"]).astype(int)

county_2017["2017_population"] = county_2017["2017_population"].astype(float)

county_2017["2017_income"] = county_2017["2017_income"].astype(float)

In [16]:
county_2017.set_index(["GEOFIPS"], inplace = True)

In [17]:
trade_employ.reset_index(inplace = True)

trade_employ = trade_employ.merge(county_2017[["2017_income","2017_population"]],
                                  left_on = "area_fips", right_index = True, how = "left")

#trade_employ.drop(labels = "index", axis = 1, inplace = True)

trade_employ.set_index(["area_fips", "time"],inplace = True)

In [18]:
file_path = cwd + "\\data\\trade_employment_blssingle19.parquet"

pq.write_table(pa.Table.from_pandas(trade_employ.reset_index()), file_path)

In [19]:
trade_employ.loc[10003].tail(10)

,total_exp_pc,china_exp_pc,tariff,emplvl_2017,fips,total_employment,emp_rtl,emp_all,emp_gds,emp_ngds,rural_share,2010_population,2017_income,2017_population
time,,,,,,,,,,,,,,
2018-09-01,514.280421,38.330528,0.260085,9072.0,10003,249775.0,29784.0,254270.0,26262.0,228008.0,0.046024,538479.0,68336.0,555036.0
2018-10-01,545.881585,39.658499,0.464673,9072.0,10003,249775.0,30244.0,257394.0,26398.0,230996.0,0.046024,538479.0,68336.0,555036.0
2018-11-01,516.246195,37.360160,0.456547,9072.0,10003,249775.0,31428.0,259963.0,26256.0,233707.0,0.046024,538479.0,68336.0,555036.0
2018-12-01,499.745029,39.971812,0.456547,9072.0,10003,249775.0,31706.0,260304.0,26435.0,233869.0,0.046024,538479.0,68336.0,555036.0
2019-01-01,496.613756,34.459205,0.448531,9072.0,10003,249775.0,30079.0,253938.0,25984.0,227954.0,0.046024,538479.0,68336.0,555036.0
2019-02-01,485.506472,35.747547,0.448519,9072.0,10003,249775.0,29736.0,253956.0,25848.0,228108.0,0.046024,538479.0,68336.0,555036.0
2019-03-01,552.995665,47.108656,0.448519,9072.0,10003,249775.0,29888.0,255809.0,26280.0,229529.0,0.046024,538479.0,68336.0,555036.0
2019-04-01,504.428530,36.677304,0.448519,9072.0,10003,249775.0,30013.0,257083.0,26451.0,230632.0,0.046024,538479.0,68336.0,555036.0
2019-05-01,525.509009,39.118929,0.448517,9072.0,10003,249775.0,29945.0,257617.0,26518.0,231099.0,0.046024,538479.0,68336.0,555036.0
